<a href="https://colab.research.google.com/github/apoorvareddy612/Capstone-Project-2025/blob/main/src/data_extractor/extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade yt_dlp
!pip install pydub SpeechRecognition ffmpeg ffprobe googletrans==3.1.0a0 transformers
!pip install google-cloud-speech yt-dlp pydub tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# import yt_dlp
# import time
# import re
# import os
# from pydub import AudioSegment
# import speech_recognition as sr
# import math
# from tqdm import tqdm
# from googletrans import Translator
# from threading import Thread


# # Example usage
# youtube_url = 'https://www.youtube.com/watch?v=kW_XDUAjFVM'
# credentials_path = '/Users/apoorvareddy/Downloads/Academic/DATS6501/compact-retina-447922-c6-2889a6e457b3.json'

# ydl_opts={}

# with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         info_dict = ydl.extract_info(youtube_url, download=False)
# video_title = info_dict['title']
# video_name = re.sub('[\\\\/*?:"<>|]', '', video_title)
# name = video_name
# ydl_opts = {
#      'format': 'm4a/bestaudio/best',
#          'noplaylist': True,
#          'continue_dl': True,
#          'outtmpl': f'/content/drive/MyDrive/Data/{name}.wav',
#          'postprocessors': [{
#              'key': 'FFmpegExtractAudio',
#              'preferredcodec': 'wav',
#              'preferredquality': '192',
#          }],
#          'geobypass':True,
#          'ffmpeg_location':'/usr/bin/ffmpeg'
#  }

# with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#      error_code = ydl.download(youtube_url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=kW_XDUAjFVM
[youtube] kW_XDUAjFVM: Downloading webpage
[youtube] kW_XDUAjFVM: Downloading ios player API JSON
[youtube] kW_XDUAjFVM: Downloading tv player API JSON
[youtube] kW_XDUAjFVM: Downloading player 6e1dd460
[youtube] kW_XDUAjFVM: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=kW_XDUAjFVM
[youtube] kW_XDUAjFVM: Downloading webpage
[youtube] kW_XDUAjFVM: Downloading ios player API JSON
[youtube] kW_XDUAjFVM: Downloading tv player API JSON
[youtube] kW_XDUAjFVM: Downloading m3u8 information
[info] kW_XDUAjFVM: Downloading 1 format(s): 140
[download] Destination: /content/drive/MyDrive/Data/Short Stuff Flower Power  STUFF YOU SHOULD KNOW.wav
[download] 100% of   11.04MiB in 00:00:00 at 23.24MiB/s  
[FixupM4a] Correcting container of "/content/drive/MyDrive/Data/Short Stuff Flower Power  STUFF YOU SHOULD KNOW.wav"
[ExtractAudio] Destination: /content/drive/MyDrive/Data/Short Stuff Flowe

In [56]:
import yt_dlp
import re
import os
from pydub import AudioSegment
from google.cloud import speech_v1 as speech
from google.oauth2 import service_account
from tqdm import tqdm
import math
import uuid

def download_audio(youtube_url, output_dir):
    unique_id = str(uuid.uuid4())
    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'noplaylist': True,
        'outtmpl': os.path.join(output_dir, f"{unique_id}"),  # Unique ID with extension
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        }],
        'ffmpeg_location': '/usr/bin/ffmpeg'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.extract_info(youtube_url, download=True)
        return os.path.join(output_dir, f"{unique_id}.mp3")


def split_audio(file_path, chunk_length_ms):
    audio = AudioSegment.from_file(file_path)
    print(audio)
    chunks = []
    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i+chunk_length_ms]
        chunks.append(chunk)

        # Calculate the size in MB
        chunk_size_mb = len(chunk.raw_data) / (1024 * 1024)
        print(f"Chunk {len(chunks)} size: {chunk_size_mb:.2f} MB")

    return chunks

# Define the function to transcribe audio using Google Speech-to-Text API
def transcribe_audio_chunk(audio_chunk, credentials_path, language="en-US"):
    credentials = service_account.Credentials.from_service_account_file(credentials_path)
    client = speech.SpeechClient(credentials=credentials)

    audio_content = audio_chunk.raw_data
    audio = speech.RecognitionAudio(content=audio_content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=48000,
        language_code=language
    )

    response = client.recognize(config=config, audio=audio)
    transcripts = [result.alternatives[0].transcript for result in response.results]
    return ' '.join(transcripts)

# Main function to process the YouTube video and transcribe
def transcribe_youtube_podcast(youtube_url, credentials_path, output_dir="/content/drive/MyDrive/Data", chunk_length_ms=10000):
    os.makedirs(output_dir, exist_ok=True)
    audio_file = download_audio(youtube_url, output_dir)

    print(f"Audio downloaded to {audio_file}")

    # Split audio into manageable chunks
    audio_chunks = split_audio(audio_file, chunk_length_ms)
    print(f"Audio split into {len(audio_chunks)} chunks.")

    # Transcribe each chunk and combine results
    full_transcript = ""
    for i, chunk in enumerate(tqdm(audio_chunks, desc="Transcribing audio chunks")):
        try:
            transcript = transcribe_audio_chunk(chunk, credentials_path)
            full_transcript += f"Chunk {i+1}:\n{transcript}\n\n"
        except Exception as e:
            print(f"Error transcribing chunk {i+1}: {e}")

    # Save the transcript to a file
    transcript_file = os.path.join(output_dir, "transcript.txt")
    with open(transcript_file, "w") as f:
        f.write(full_transcript)

    print(f"Transcription complete. Transcript saved to {transcript_file}")

# Example usage
youtube_url = 'https://www.youtube.com/watch?v=kW_XDUAjFVM'
credentials_path = '/content/drive/MyDrive/compact-retina-447922-c6-2889a6e457b3.json'
transcribe_youtube_podcast(youtube_url, credentials_path)

[youtube] Extracting URL: https://www.youtube.com/watch?v=kW_XDUAjFVM
[youtube] kW_XDUAjFVM: Downloading webpage
[youtube] kW_XDUAjFVM: Downloading ios player API JSON
[youtube] kW_XDUAjFVM: Downloading tv player API JSON
[youtube] kW_XDUAjFVM: Downloading m3u8 information
[info] kW_XDUAjFVM: Downloading 1 format(s): 140
[download] Destination: /content/drive/MyDrive/Data/9d483643-b3b4-45fb-b289-acbbb436a711
[download] 100% of   11.04MiB in 00:00:00 at 29.79MiB/s  
[FixupM4a] Correcting container of "/content/drive/MyDrive/Data/9d483643-b3b4-45fb-b289-acbbb436a711"
[ExtractAudio] Destination: /content/drive/MyDrive/Data/9d483643-b3b4-45fb-b289-acbbb436a711.mp3
Deleting original file /content/drive/MyDrive/Data/9d483643-b3b4-45fb-b289-acbbb436a711 (pass -k to keep)
Audio downloaded to /content/drive/MyDrive/Data/9d483643-b3b4-45fb-b289-acbbb436a711.mp3
Chunk 1 size: 1.68 MB
Chunk 2 size: 1.68 MB
Chunk 3 size: 1.68 MB
Chunk 4 size: 1.68 MB
Chunk 5 size: 1.68 MB
Chunk 6 size: 1.68 MB
Chun

Transcribing audio chunks: 100%|██████████| 72/72 [05:28<00:00,  4.56s/it]

Transcription complete. Transcript saved to /content/drive/MyDrive/Data/transcript.txt
